In [4]:
import pandas as pd
import numpy as np
import json
from clearml import Task, Logger


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

SEED = 21

In [3]:
%env CLEARML_WEB_HOST=https://app.clear.ml
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml
# mlsd
%env CLEARML_API_ACCESS_KEY=X2FEZVG1P5PO793YKWK0
%env CLEARML_API_SECRET_KEY=lzo5y1tNKBQHe7qGdfF4oEde5vhRiNGknrH2yGSx50LBkwXm3O

env: CLEARML_WEB_HOST=https://app.clear.ml
env: CLEARML_API_HOST=https://api.clear.ml
env: CLEARML_FILES_HOST=https://files.clear.ml
env: CLEARML_API_ACCESS_KEY=X2FEZVG1P5PO793YKWK0
env: CLEARML_API_SECRET_KEY=lzo5y1tNKBQHe7qGdfF4oEde5vhRiNGknrH2yGSx50LBkwXm3O


In [5]:
task = Task.init(
    project_name='ML_SD', 
    task_name='test_platform', 
    tags=['logreg','tfidf'])

ClearML Task: created new task id=2159a8990e82491d9cd9237b8d4d2be9
ClearML results page: https://app.clear.ml/projects/922c69dbd48249b183708fef50f18e10/experiments/2159a8990e82491d9cd9237b8d4d2be9/output/log


ClearML Monitor: GPU monitoring failed getting GPU reading, switching off GPU monitoring
ClearML Monitor: Could not detect iteration reporting, falling back to iterations as seconds-from-start


In [7]:
fpath = 'data/RuMedPrimeData.tsv'
df = pd.read_csv(fpath, sep='\t')
task.upload_artifact(name='data.raw', artifact_object=fpath)

True

In [8]:
df.head()

,symptoms,anamnesis,icd10,new_patient_id,new_event_id,new_event_time
0,"Сухость кожи, мышечная слабость, нарушение мен...",Месяц назад сильный стресс и на этом фоне появ...,E01.8,qf156c36,q5fc2cb1,2027-05-19
1,"Жалобы ГБ: головная боль, головокружение на фо...","Начало острое, постепенное, после переохлажден...",J06.9,q9321cf8,qe173f20,2023-03-24
2,"головная боль, озноб, насморк, легкая заложенн...","Начало острое, *ДАТА* после переохлаждения. Пр...",J00,qcbc06c5,q653efaa,2026-12-22
3,"На изжога периодически , отрыжка воздухом, . б...",Впервые пояились вышеизложенные жалобы в течен...,K21.0,qfd53b1f,qe48681b,2026-01-15
4,"- лабильность настроения, плаксивость.","Тиреоидэктомия в*ДАТА* по поводу МУЗ, по гисто...",E89.0,q783a54b,q4109cab,2021-05-28


## Данные

In [9]:
df['icd10'] = df['icd10'].apply(lambda s: s.split('.')[0])
df = df[['symptoms', 'icd10']].reset_index(drop=True)

In [10]:
df

,symptoms,icd10
0,"Сухость кожи, мышечная слабость, нарушение мен...",E01
1,"Жалобы ГБ: головная боль, головокружение на фо...",J06
2,"головная боль, озноб, насморк, легкая заложенн...",J00
3,"На изжога периодически , отрыжка воздухом, . б...",K21
4,"- лабильность настроения, плаксивость.",E89
...,...,...
7620,"Насморк,затек слизи по задней стенке.Высыпания...",J30
7621,"на одышку в покое, при незначительной физическ...",J45
7622,на периодическую боль в поясничной области,M54
7623,"Боли в левом плечевом суставе, усилив. в полож...",M19


In [11]:
parts = np.split(df.sample(frac=1, random_state=SEED), [int(0.735*len(df)), int(0.8675*len(df))])

In [12]:
train_data = parts[0]
dev_data = parts[1]
test_data = parts[2]

In [13]:
task.upload_artifact(name='data.train', artifact_object=train_data)
task.upload_artifact(name='data.val', artifact_object=dev_data)
task.upload_artifact(name='data.test', artifact_object=test_data)

True

In [14]:
icd_freq = (train_data['icd10'].value_counts() > 10)
labels = [k for k, v in dict(icd_freq).items() if v]

In [15]:
train_data = train_data[train_data['icd10'].isin(labels)]
dev_data = dev_data[dev_data['icd10'].isin(labels)]
test_data = test_data[test_data['icd10'].isin(labels)]

In [16]:
task.upload_artifact(name='data.train', artifact_object=train_data)
task.upload_artifact(name='data.val', artifact_object=dev_data)
task.upload_artifact(name='data.test', artifact_object=test_data)

True

In [ ]:
# df_list = [train_data, dev_data, test_data]
# for i in range(0, 3):
#     print(df_list[i].shape)
#     print(df_list[i]['icd10'].value_counts())

## TFIDF и LogReg

In [17]:
def hit_at_n(y_true, y_pred, n=3):
    assert len(y_true) == len(y_pred)
    hit_count = 0
    for l, row in zip(y_true, y_pred):
        order = (np.argsort(row)[::-1])[:n]
        hit_count += int(l in order)
    return round(hit_count / float(len(y_true)) * 100, 2)


def encode_text(tfidf, text_data, labels, l2i, mode='train'):
    if mode == 'train':
        X = tfidf.fit_transform(text_data)
    else:
        X = tfidf.transform(text_data)
    y = labels.map(l2i)
    return X, y


def logits2codes(logits, i2l, n=3):
    codes = []
    for row in logits:
        order = np.argsort(row)[::-1]
        codes.append([i2l[i] for i in order[:n]])
    return codes

In [18]:
text_id = 'symptoms'
label_id = 'icd10' 
index_id = 'idx'

i2l = dict(enumerate(sorted(train_data[label_id].unique())))
l2i = {label: i for i, label in i2l.items()}

In [19]:
tfidf = TfidfVectorizer(analyzer='char', ngram_range=(3, 8))
clf = LogisticRegression(penalty='l2', C=10, multi_class='ovr', n_jobs=10, verbose=1)

X, y = encode_text(tfidf, train_data[text_id], train_data[label_id], l2i)

In [ ]:
# print(X.shape)
# print(tfidf.get_feature_names_out()[::2000])

In [ ]:
clf.fit(X, y)

In [21]:
X_val, y_val = encode_text(tfidf, dev_data[text_id], dev_data[label_id], l2i, mode='val')
y_val_pred = clf.predict_proba(X_val)

accuracy = hit_at_n(y_val, y_val_pred, n=1)
hit_3 = hit_at_n(y_val, y_val_pred, n=3)
print (f'\nscores on dev set: {accuracy} / {hit_3}')


scores on dev set: 49.04 / 74.4


In [ ]:
X_test, _ = encode_text(tfidf, test_data[text_id], test_data[label_id], l2i, mode='test')
y_test_pred = clf.predict_proba(X_test)

accuracy = hit_at_n(_, y_test_pred, n=1)
hit_3 = hit_at_n(_, y_test_pred, n=3)
print (f'\nscores on test set: {accuracy} / {hit_3}')

test_codes = logits2codes(y_test_pred, i2l)

recs = []
for true, pred in zip(test_data[label_id], test_codes):
    recs.append({label_id: true, 'prediction': pred})


scores on test set: 49.29 / 73.94


In [ ]:
out_fname = 'results.jsonl'
with open(out_fname, 'w') as fw:
    for rec in recs:
        json.dump(rec, fw, ensure_ascii=False)
        fw.write('\n')